In [97]:
import numpy as np

class KeyGenerator:
    def __init__(self, full=False, seed=None) -> None:
        ascii_vocab = [(48, 57), (65, 90), (97, 122)]
        vocab = [chr(i) for g in [range(a, b+1) for a, b in ascii_vocab] for i in g]
        if full: vocab += [c for c in "-_"]
        self.vocab = vocab
        self._generator = np.random.default_rng(seed)

    def GenerateUID(self, l:int=8, blacklist: set[str]=set()) -> str:
        key: str|None = None
        while key is None or key in blacklist:
            digits = self._generator.integers(0, len(self.vocab), l)
            key = "".join([self.vocab[i] for i in digits])
        return key
    
    def FromInt(self, i: int, l: int=8, little_endian=False):
        chunks = [self.vocab[0]]*l
        place = 0
        while i > 0 and place < l:
            chunk_k = i % len(self.vocab)
            i = (i - chunk_k) // len(self.vocab)
            chunks[place] = self.vocab[chunk_k]
            place += 1
        if not little_endian: chunks.reverse()
        return "".join(chunks)
    
    def FromHex(self, hex: str, l: int=8, little_endian=False):
        return self.FromInt(int(hex, 16), l, little_endian)

In [98]:
from __future__ import annotations
from dataclasses import dataclass, field
import os, sys
from typing import Any, Generator, Iterable, Literal
import hashlib
import numpy as np
import json
from collections import deque
from pathlib import Path

class Namespace:
    def __init__(self) -> None:
        self.node_signatures: dict[int, str] = {}
        self._last_k: int = 0
        self._kg = KeyGenerator(True)
        self._KLEN = 4
        self._MAX_K = len(self._kg.vocab)**self._KLEN

    def NewKey(self):
        self._last_k += 1
        assert self._last_k < self._MAX_K
        return self._last_k, self._kg.FromInt(self._last_k, self._KLEN)

class Hashable:
    def __init__(self, ns: Namespace) -> None:
        self.namespace = ns
        self.hash, self.key = ns.NewKey()

    def __hash__(self) -> int:
        return self.hash
    
    def __eq__(self, __value: object) -> bool:
        K = "key"
        return hasattr(__value, K) and self.key == getattr(__value, K)

class Node(Hashable):
    def __init__(
        self,
        ns: Namespace,
        properties: set[str],
        parents: set[Node],
    ) -> None:
        super().__init__(ns)
        self.namespace = ns
        self.properties = properties
        self.parents = parents
        self._sig: str|None = None
        # self._diffs = set()
        # self._sames = set()

    def __str__(self) -> str:
        return f"({'-'.join(self.properties)}:{self.key})"

    def __repr__(self) -> str:
        return f"{self}"
    
    def IsA(self, other: Node) -> bool:
        # if other.key in self._diffs: return False
        # if other.key in self._sames: return True
        if not other.properties.issubset(self.properties):
            # self._diffs.add(other.key)
            return False
        # self._sames.add(other.key)
        # if compare_lineage: return  other.parents.issubset(self.parents)
        return True

    def Signature(self):
        if self._sig is None:
            psig = ",".join(sorted(p.Signature() for p in self.parents))
            sig = ",".join(sorted(self.properties))
            self._sig = f'{sig}:[{psig}]' if len(self.parents)>0 else sig
        return self._sig

class Dependency(Node):
    def __init__(self, namespace: Namespace, properties: set[str], parents: set[Node]) -> None:
        super().__init__(namespace, properties, parents)

    def __str__(self) -> str:
        return f"(D:{'-'.join(self.properties)})"
    
class Endpoint(Node):
    def __init__(self, namespace: Namespace, properties: set[str], parents: dict[Endpoint, Node]=dict()) -> None:
        super().__init__(namespace, properties, set(parents))
        self._parent_map = parents # real, proto

    def Iterparents(self):
        """real, prototype"""
        for e, p in self._parent_map.items():
            yield e, p

class Transform(Hashable):
    def __init__(self, ns: Namespace) -> None:
        super().__init__(ns)
        self.requires: list[Dependency] = list()
        self.produces: list[Dependency] = list()
        self.deletes: set[Dependency] = set()
        self._ns = ns
        self._input_group_map: dict[int, list[Dependency]] = {}
        self._key = ns.NewKey()
        self._seen: set[str] = set()

    def __str__(self) -> str:
        def _props(d: Dependency):
            return "{"+"-".join(d.properties)+"}"
        return f"{','.join(_props(r) for r in self.requires)}->{','.join(_props(p) for p in self.produces)}"

    def __repr__(self): return f"{self}"

    def AddRequirement(self, properties: Iterable[str], parents: set[Dependency]=set()):
        prototype = Dependency(properties=set(properties), parents=parents, namespace=self._ns)
        return self._add_dependency(self.requires, prototype)

    def AddProduct(self, properties: Iterable[str], parents: set[Dependency]=set()):
        prototype = Dependency(properties=set(properties), parents=parents, namespace=self._ns)
        for d in self.deletes:
            assert not prototype.IsA(d), f"can not produce and delete product[{d}]"
        return self._add_dependency(self.produces, prototype)
    
    # def AddDeletion(self, to_delete: Dependency):
    #     print("warning!, deletion not implemented")
    #     assert to_delete in self.requires, f"{to_delete} not in requirements"
    #     if to_delete in self.deletes: return # already added
    #     for p in self.produces:
    #         assert not p.IsA(to_delete), f"can not produce and delete product [{p}]"
    #     self.deletes.add(to_delete)

    def _add_dependency(self, destination: list[Dependency], prototype: Dependency):
        # _dep = Dependency(properties=set(properties), parents=_parents, namespace=self._ns)
        _dep = prototype
        _parents = _dep.parents
        # assert not any(e.IsA(_dep) for e in destination), f"prev. dep ⊆ new dep"
        # assert not any(_dep.IsA(e) for e in destination), f"new dep ⊆ prev. dep "
        # destination.add(_dep)
        destination.append(_dep)
        if destination == self.requires:
            i = len(self.requires)-1
            for p in _parents:
                assert p in self.requires, f"{p} not added as a requirement"
            self._input_group_map[i] = self._input_group_map.get(i, [])+list(_parents)
        return _dep

    def _sig(self, endpoints: Iterable[Endpoint]):
        # return "".join(e.key for e in endpoints)
        return self.key+"-"+ "".join(e.key for e in endpoints)

    # just all possibilities regardless of lineage
    def Possibilities(self, have: set[Endpoint], constraints: dict[Dependency, Endpoint]=dict()) -> Generator[list[Endpoint], Any, None]:
        matches: list[list[Endpoint]] = []
        constraints_used = False
        for req in self.requires:
            if req in constraints:
                must_use = constraints[req]
                _m = [must_use]
            else:
                _m = [m for m in have if m.IsA(req)]
            if len(_m) == 0: return None
            matches.append(_m)
        if len(constraints)>0 and not constraints_used: return None

        indexes = [0]*len(matches)
        indexes[0] = -1
        def _advance():
            i = 0
            while True:
                indexes[i] += 1
                if indexes[i] < len(matches[i]): return True
                indexes[i] = 0
                i += 1
                if i >= len(matches): return False
        while _advance():
            yield [matches[i][j] for i, j in enumerate(indexes)]
    
    # filter possibilities based on correct lineage
    def Valids(self, matches: Iterable[list[Endpoint]]):
        black_list: set[tuple[int, Endpoint]] = set()
        white_list: set[tuple[int, Endpoint]] = set()

        choosen: list[Endpoint] = []
        for config in matches:
            ok = True
            for i, (e, r) in enumerate(zip(config, self.requires)):
                k = (i, e)
                if k in black_list: ok=False; break
                if k in white_list: continue
                
                parents = self._input_group_map.get(i, [])
                if len(parents) == 0: # no lineage req.
                    white_list.add(k)
                    continue
                
                for prototype in parents:
                    # parent must already be in choosen, since it must have been added
                    # as a req. before being used as a parent during setup
                    found = False
                    for p in choosen:
                        if not p.IsA(prototype): continue
                        if p in e.parents: found=True; break
                    if not found: black_list.add(k); ok=False; break
                if not ok: break
            if ok: yield config

    def Apply(self, inputs: Iterable[tuple[Endpoint, Node]]):
        # deleted = {}
        # for r, (e, e_proto) in zip(self.requires, inputs):
        #     assert e.IsA(r), f"{e_proto}, {e}, {r}"
        #     if r in self.deletes: deleted[e] = e_proto

        inputs_dict = dict(inputs)
        parent_dict: dict[Any, Any] = {}
        for e, _ in inputs_dict.items():
            for p, pproto in e.Iterparents():
                if p in parent_dict: continue
                parent_dict[p] = pproto
        for e, eproto in inputs_dict.items():
            parent_dict[e] = eproto
        produced = {
            Endpoint(
                namespace=self._ns,
                properties=out.properties,
                parents=parent_dict
            ):out
        for out in self.produces}
        # return Application(self, inputs_dict, produced, deleted)
        return Application(self, inputs_dict, produced)

@dataclass
class Application:
    transform: Transform
    used: dict[Endpoint, Node]
    produced: dict[Endpoint, Dependency]
    # deleted: dict[Endpoint, Node]

    def __str__(self) -> str:
        # return f"{self.transform} || {','.join(str(e) for e in self.used.keys())} -> {','.join(str(e) for e in self.produced)} |x {','.join(str(e) for e in self.deleted)}"
        return f"{self.transform} || {','.join(str(e) for e in self.used.keys())}->{','.join(str(e) for e in self.produced)}"

    def __repr__(self) -> str:
        return f"{self}"

@dataclass
class HasSteps:
    steps: int

@dataclass
class Result(HasSteps):
    application: Application
    dependency_plan: list[Application]
    
@dataclass
class DepResult(HasSteps):
    plan: list[Application]
    endpoint: Endpoint

def Solve(given: Iterable[Endpoint], target: Transform, transforms: Iterable[Transform], _debug=False):
    @dataclass
    class State:
        have: dict[Endpoint, Dependency]
        needed: set[Dependency]
        target: Dependency|Transform
        lineage_requirements: dict[Node, Endpoint]
        seen_signatures: set[str]
        depth: int

    def _get_producers_of(target: Dependency):
        for tr in transforms:
            for p in tr.produces:
                if p.IsA(target):
                    yield tr
                    break

    if _debug:
        log_path = Path("./cache/debug_log.txt")
        log_path.parent.mkdir(parents=True, exist_ok=True)
        log = open("./cache/debug_log.txt", "w")
        debug_print = lambda *args: log.write(" ".join(str(a) for a in args)+"\n") if args[0] != "END" else log.close()
    else:
        debug_print = lambda *args: None

    _apply_cache: dict[str, Application] = {}
    def _apply(target: Transform, inputs: Iterable[tuple[Endpoint, Node]]):
        sig  = "".join(e.key+d.key for e, d in inputs)
        if sig in _apply_cache:
            return _apply_cache[sig]
        appl = target.Apply(inputs)
        _apply_cache[sig] = appl
        return appl

    def _satisfies_lineage(tproto: Dependency, candidate: Endpoint):
        for tp_proto in tproto.parents:
            if all(not p.IsA(tp_proto) for p, _ in candidate.Iterparents()):
                return False
        return True

    HORIZON=64
    def _solve_dep(s: State) -> list[DepResult]:
        if s.depth >= HORIZON:
            if _debug: debug_print(f" <-  HORIZON", s.depth)
            return []
        target: Dependency = s.target
        assert isinstance(target, Dependency), f"{s.target}, not dep"
        if _debug: debug_print(f" ->", s.target, s.lineage_requirements)
        if _debug: debug_print(f"   ", s.have.keys())

        candidates:list[DepResult] = []
        for e, eproto in s.have.items():
            if not e.IsA(target): continue
            acceptable = True
            for rproto, r in s.lineage_requirements.items():
                if e == r: continue
                if eproto.IsA(rproto): # e is protype, but explicitly breaks lineage
                    acceptable=False; break

                for p, pproto in e.Iterparents():
                    if rproto.IsA(pproto):
                        if p != r:
                            acceptable=False; break

            if not acceptable:
                continue
            else:
                if _debug: debug_print(f"    ^candidate", e, eproto, e.parents)
                if _debug: debug_print(f"    ^reqs.    ", s.lineage_requirements)
                candidates.append(DepResult(0, [], e))
            # elif quality == 2:
            #     if DEBUG: debug_print(f" <-", s.target, e, "DIRECT")
            #     return [DepResult(0, [], e)]

        def _add_result(res: Result):
            ep: Endpoint|None = None
            for e in res.application.produced:
                if e.IsA(target):
                    ep = e; break
            assert isinstance(ep, Endpoint)
            if not _satisfies_lineage(target, ep): return
            candidates.append(DepResult(
                res.steps,
                res.dependency_plan+[res.application],
                ep,
            ))

        for tr in _get_producers_of(target):
            # if target in tr.deletes: continue
            results = _solve_tr(State(s.have, s.needed, tr, s.lineage_requirements, s.seen_signatures, s.depth))
            for res in results:
                _add_result(res)

        if _debug: debug_print(f" <-", s.target, f"{len(candidates)} sol.", candidates[0].endpoint if len(candidates)>0 else None)
        return candidates

    _transform_cache: dict[str, list[Result]] = {}
    def _solve_tr(s: State) -> list[Result]:
        assert isinstance(s.target, Transform), f"{s.target} not tr"
        target: Transform = s.target
        if _debug: debug_print(f">>>{s.depth:02}", s.target, s.lineage_requirements)
        for h in s.have:
            if _debug: debug_print(f"      ", h)

        # memoization
        sig = "".join(e.key for e in s.have)
        sig += f":{s.target.key}"
        sig += ":"+"".join(e.key for e in s.lineage_requirements.values())
        if sig in _transform_cache:
            if _debug: debug_print(f"<<<{s.depth:02} CACHED: {len(_transform_cache[sig])} solutions")
            return _transform_cache[sig]
        if sig in s.seen_signatures:
            if _debug: debug_print(f"<<<{s.depth:02} FAIL: is loop")
            return []

        plans: list[list[DepResult]] = []
        for i, req in enumerate(s.target.requires):
            req_p = {}
            for proto, e in s.lineage_requirements.items():
                if req.IsA(proto): continue
                req_p[proto] = e

            results = _solve_dep(State(s.have, s.needed|{req}, req, req_p, s.seen_signatures|{sig}, s.depth+1))
            
            if len(results) == 0:
                if _debug: debug_print(f"<<< FAIL", s.target, req)
                return []
            else:
                plans.append(results)

        def _gather_valid_inputs():
            valids: list[list[DepResult]] = []
            ii = 0
            def _gather(req_i: int, req: Dependency, res: DepResult, deps: dict, used: set[Endpoint], inputs: list[DepResult]):
                nonlocal ii; ii += 1         
                if _debug: debug_print(f"          ", deps)
                if _debug: debug_print(f"    ___", req, req.parents)
                if _debug: debug_print(f"        __", res.endpoint, list(res.endpoint.Iterparents()))
                if res.endpoint in used:
                    if _debug: debug_print(f"    ___ FAIL: duplicate input", res.endpoint)
                    return
                # used.add(res.endpoint)

                if not _satisfies_lineage(req, res.endpoint):
                    if _debug: debug_print(f"    ___ FAIL: unsatisfied lineage", req)
                    return

                for rproto in req.parents:
                    r = deps[rproto]
                    # if all(not p.IsA(rproto) for p, pproto in res.endpoint.Iterparents()):
                    #     if DEBUG: debug_print(f"    ___ FAIL: unsatisfied lineage", rproto)
                    #     _fail=True; break
                    res_parents = list(res.endpoint.Iterparents())
                    res_parents.reverse()
                    for p, pproto in res_parents:
                        if not p.IsA(rproto): continue
                        if p!=r:
                            if _debug: debug_print(f"    ___ FAIL: lineage mismatch", p, r)
                            return
                        else:
                            break # in the case of asm -> bin, the closest ancestor takes priority
                # deps[req] = res.endpoint

                if req_i >= len(target.requires)-1:
                    valids.append(inputs+[res])
                else:
                    req_i += 1
                    for i, next_res in enumerate(plans[req_i]):
                        _gather(req_i, target.requires[req_i], next_res, deps|{req:res.endpoint}, used|{res.endpoint}, inputs+[res])
            req_i = 0
            for i, next_res in enumerate(plans[req_i]):
                _gather(0, target.requires[req_i], next_res, {}, set(), [])
            total = 1
            for s in plans:
                total *= len(s)
            if _debug: debug_print(f"    ## {ii} visited, {total} combos")
            return valids

        if _debug: debug_print(f"<<<{s.depth:02}", s.target, s.lineage_requirements)
        if _debug: debug_print(f"     ", [len(x) for x in plans])
        solutions: list[Result] = []
        # for inputs in _iter_satisfies():
        for inputs in _gather_valid_inputs():
            my_appl = _apply(s.target, [(res.endpoint, req) for req, res in zip(s.target.requires, inputs)])
            consolidated_plan: list[Application] = []
            produced_sigs: set[str] = {p.Signature() for p in my_appl.produced}
            # if DEBUG: debug_print(f"   __", my_appl)
            for res in inputs:
                for appl in res.plan:
                    if all(p.Signature() in produced_sigs for p in appl.produced): continue
                    consolidated_plan.append(appl)
                    produced_sigs = produced_sigs.union(p.Signature() for p in appl.produced)
            solutions.append(Result(
                len(consolidated_plan),
                my_appl,
                consolidated_plan,
            ))
            # if DEBUG: debug_print(f"    *", my_appl)
            # if DEBUG: debug_print(f"     ", [res.endpoint for res in inputs])
            # if DEBUG: debug_print(f"    .", target.requires)
            # for appl in consolidated_plan:
            #     if DEBUG: debug_print(f"    __", appl)
        if _debug: debug_print(f"     ", f"{len(solutions)} sol.", solutions[0].application.produced if len(solutions)>0 else None)
        solutions = sorted(solutions, key=lambda s: s.steps)
        _transform_cache[sig] = solutions
        return solutions

    input_tr = Transform(target._ns)
    given_dict = {g:input_tr.AddProduct(g.properties) for g in given}
    res = _solve_tr(State(given_dict, set(), target, {}, set(), 0))
    if _debug: debug_print("END")
    return res


def _set(s: str):
    return set(s.split(", "))
 
transforms = []
NS = Namespace()

# t = Transform(NS)
# t.AddRequirement(_set("dna"))
# t.AddProduct(_set("contigs, asm, annable"))
# transforms.append(t)

# t = Transform(NS)
# r = t.AddRequirement(_set("dna"))
# t.AddRequirement(_set("contigs, asm"), {r})
# t.AddProduct(_set("contigs, bin, annable"))
# transforms.append(t)

# t = Transform(NS)
# t.AddRequirement(_set("db"))
# t.AddRequirement(_set("annable"))
# t.AddProduct(_set("ann"))
# transforms.append(t)

# t = Transform(NS)
# ann = t.AddRequirement(_set("annable"))
# r = t.AddRequirement(_set("db, cog"))
# t.AddRequirement(_set("ann"), {r, ann})
# r = t.AddRequirement(_set("db, kegg"))
# t.AddRequirement(_set("ann"), {r, ann})
# t.AddProduct(_set("table"))
# transforms.append(t)

# t = Transform(NS)
# db1 = t.AddRequirement(_set("db, cog"))
# db2 = t.AddRequirement(_set("db, kegg"))
# asm = t.AddRequirement(_set("contigs, asm"))
# bin = t.AddRequirement(_set("contigs, bin"))
# # t.AddRequirement(_set("ann"), {asm, db1})
# # t.AddRequirement(_set("ann"), {asm, db2})
# # t.AddRequirement(_set("ann"), {bin, db1})
# # t.AddRequirement(_set("ann"), {bin, db2})
# t.AddRequirement(_set("table"), {asm, db1, db2})
# t.AddRequirement(_set("table"), {bin, db1, db2})
# t.AddProduct(_set("figure"))
# transforms.append(t)

# t = Transform(NS)
# t.AddRequirement(_set("precog"))
# t.AddProduct(_set("db, cog"))
# transforms.append(t)

# t = Transform(NS)
# t.AddRequirement(_set("prekegg"))
# t.AddProduct(_set("db, kegg"))
# transforms.append(t)

t = Transform(NS)
d = t.AddRequirement(_set("a"))
t.AddProduct(_set("b"))
# t.AddDeletion(d)
transforms.append(t)

t = Transform(NS)
d = t.AddRequirement(_set("b"))
t.AddProduct(_set("c"))
# t.AddDeletion(d)
transforms.append(t)

t = Transform(NS)
d = t.AddRequirement(_set("b"))
t.AddProduct(_set("x"))
transforms.append(t)

haves = [Endpoint(NS, _set(r)) for r in [
    # "precog",
    # # "db, cog",
    # "db, kegg",
    "a",
]]

target = Transform(NS)
r = target.AddRequirement(_set("c"))
r = target.AddRequirement(_set("x"))
# db = target.AddRequirement(_set("cog"))
# target.AddRequirement(_set("ann"), {db, r})
# target.AddRequirement(_set("ann"), {db})

# r = target.AddRequirement(_set("bin"))
# target.AddRequirement(_set("table"), {r})

# target.AddRequirement(_set("figure"))

def _print_sol(plan: Result):
    for i, step in enumerate(plan.dependency_plan):
        print(f"step {i+1}")
        print(step.transform)
        print("used")
        for x in step.used:
            print(" ", x)
        print("produced")
        for x in step.produced:
            print(" ", x)
        # if len(step.deleted) > 0:
        #     print("deleted")
        #     for x in step.deleted:
        #         print(" ", x)
        print()

solutions = None
def _test():
    global solutions
    solutions = Solve(haves, target, transforms, _debug=True)
    print(f"{len(solutions)} solutions")
    for res in solutions:
        _print_sol(res)
        # print(res.steps)
        # return f"{len(solutions)} solutions", res.dependency_plan+[res.application]
    print("//")
    # if res is not None:
_test()

1 solutions
step 1
{a}->{b}
used
  (a:000D)
produced
  (b:000L)

step 2
{b}->{c}
used
  (b:000L)
produced
  (c:000M)

step 3
{b}->{x}
used
  (b:000L)
produced
  (x:000N)

//


In [99]:
for res in solutions:
    for a in res.dependency_plan:
        print(a)
    print(res.application)
    print()

{a}->{b} || (a:000D)->(b:000L)
{b}->{c} || (b:000L)->(c:000M)
{b}->{x} || (b:000L)->(x:000N)
{c},{x}-> || (c:000M),(x:000N)->



In [100]:
transforms

[{a}->{b}, {b}->{c}, {b}->{x}]

In [86]:
transforms = []
NS = Namespace()

t = Transform(NS)
t.AddRequirement({"dna", "raw reads"})
t.AddProduct({"assembly", "contigs"})
transforms.append(t)

t = Transform(NS)
reads = t.AddRequirement({"dna", "raw reads"})
t.AddRequirement({"contigs", "assembly"}, parents={reads})
t.AddProduct({"binned", "contigs"})
transforms.append(t)


have = [
    Endpoint(NS, {"dna", "raw reads", "sra", "metagenomic"}),
]

target = Transform(NS)
target.AddRequirement({"binned", "contigs"})

plan = Solve(have, target, transforms)[0] # first solution

In [87]:
_print_sol(plan)

step 1
{raw reads-dna}->{assembly-contigs}
used
  (raw reads-sra-metagenomic-dna:000A)
produced
  (assembly-contigs:000H)

step 2
{raw reads-dna},{assembly-contigs}->{binned-contigs}
used
  (raw reads-sra-metagenomic-dna:000A)
  (assembly-contigs:000H)
produced
  (binned-contigs:000I)



In [88]:
transforms = []
NS = Namespace()

t = Transform(NS)
t.AddRequirement({"genomic", "contigs"})
t.AddProduct({"orfs"})
transforms.append(t)

t = Transform(NS)
t.AddRequirement({"protein reference"})
t.AddRequirement({"orfs"})
t.AddProduct({"annotations"})
transforms.append(t)

have = [
    Endpoint(NS, {"genomic", "contigs"}),
    Endpoint(NS, {"protein reference", "KEGG"}),
    Endpoint(NS, {"protein reference", "COG"}),
    Endpoint(NS, {"protein reference", "metacyc"}),
]

target = Transform(NS)
kegg = target.AddRequirement({"protein reference", "KEGG"})
metacyc = target.AddRequirement({"protein reference", "metacyc"})
target.AddRequirement({"annotations"}, {kegg})
target.AddRequirement({"annotations"}, {metacyc})

plan = Solve(have, target, transforms)[0]
_print_sol(plan)

step 1
{genomic-contigs}->{orfs}
used
  (genomic-contigs:000A)
produced
  (orfs:000Q)

step 2
{protein reference},{orfs}->{annotations}
used
  (protein reference-KEGG:000B)
  (orfs:000Q)
produced
  (annotations:000R)

step 3
{protein reference},{orfs}->{annotations}
used
  (protein reference-metacyc:000D)
  (orfs:000Q)
produced
  (annotations:000T)



In [89]:
transforms = []
NS = Namespace()

t = Transform(NS)
t.AddRequirement(_set("dna"))
t.AddProduct(_set("contigs, assembly, genomic"))
transforms.append(t)

t = Transform(NS)
r = t.AddRequirement(_set("dna"))
t.AddRequirement(_set("contigs, assembly"), {r})
t.AddProduct(_set("contigs, bin, genomic"))
transforms.append(t)

t = Transform(NS)
t.AddRequirement(_set("reference"))
t.AddRequirement(_set("genomic"))
t.AddProduct(_set("annotation"))
transforms.append(t)

t = Transform(NS)
genome = t.AddRequirement(_set("genomic"))
ref_cog = t.AddRequirement(_set("reference, COG"))
t.AddRequirement(_set("annotation"), {ref_cog, genome})
ref_kegg = t.AddRequirement(_set("reference, KEGG"))
t.AddRequirement(_set("annotation"), {ref_kegg, genome})
t.AddProduct(_set("table"))
transforms.append(t)

t = Transform(NS)
ref_cog = t.AddRequirement(_set("reference, COG"))
ref_kegg = t.AddRequirement(_set("reference, KEGG"))
asm_genome = t.AddRequirement(_set("contigs, assembly"))
bin_genome = t.AddRequirement(_set("contigs, bin"))
t.AddRequirement(_set("table"), {asm_genome, ref_cog, ref_kegg})
t.AddRequirement(_set("table"), {bin_genome, ref_cog, ref_kegg})
t.AddProduct(_set("summary figure"))
transforms.append(t)

have = [
    Endpoint(NS, {"dna", "raw reads"}),
    Endpoint(NS, {"reference", "COG"}),
    Endpoint(NS, {"reference", "KEGG"}),
]

target = Transform(NS)
target.AddRequirement({"summary figure"})

plan = Solve(have, target, transforms)[0]
_print_sol(plan)

step 1
{dna}->{assembly-genomic-contigs}
used
  (raw reads-dna:000W)
produced
  (assembly-genomic-contigs:000h)

step 2
{dna},{assembly-contigs}->{bin-genomic-contigs}
used
  (raw reads-dna:000W)
  (assembly-genomic-contigs:000h)
produced
  (bin-genomic-contigs:000i)

step 3
{reference},{genomic}->{annotation}
used
  (COG-reference:000X)
  (assembly-genomic-contigs:000h)
produced
  (annotation:000j)

step 4
{reference},{genomic}->{annotation}
used
  (KEGG-reference:000Y)
  (assembly-genomic-contigs:000h)
produced
  (annotation:000l)

step 5
{genomic},{COG-reference},{annotation},{KEGG-reference},{annotation}->{table}
used
  (assembly-genomic-contigs:000h)
  (COG-reference:000X)
  (annotation:000j)
  (KEGG-reference:000Y)
  (annotation:000l)
produced
  (table:000n)

step 6
{reference},{genomic}->{annotation}
used
  (COG-reference:000X)
  (bin-genomic-contigs:000i)
produced
  (annotation:000k)

step 7
{reference},{genomic}->{annotation}
used
  (KEGG-reference:000Y)
  (bin-genomic-contigs

In [90]:
transforms = []
NS = Namespace()

t = Transform(NS) # assembly
t.AddRequirement({"dna", "raw reads"})
t.AddProduct({"assembly", "contigs"})
transforms.append(t)

t = Transform(NS) # fosmid assembly
t.AddRequirement({"dna", "raw reads", "fosmid"})
t.AddRequirement({"dna", "end seq."})
t.AddProduct({"fosmids", "assembly", "contigs"})
transforms.append(t)

t = Transform(NS) # binning
reads = t.AddRequirement({"dna", "raw reads"})
t.AddRequirement({"contigs", "assembly"}, parents={reads})
t.AddProduct({"binned", "contigs"})
transforms.append(t)

t = Transform(NS) # genomeQC
t.AddRequirement({"binned", "contigs"})
t.AddProduct({"binned", "contigs", "med quality"})
t.AddProduct({"taxonomy", "GTDB-TK"})
t.AddProduct({"table", "checkM stats"})
transforms.append(t)

t = Transform(NS) # annotation
t.AddRequirement({"assembly", "contigs"})
t.AddProduct({"orfs"})
t.AddProduct({"annotations", "metapathways"})
transforms.append(t)

t = Transform(NS) # taxonomy
t.AddRequirement({"contigs"})
t.AddProduct({"taxonomy", "GTDB-TK"})
transforms.append(t)

have = [
    Endpoint(NS, {"dna", "raw reads", "metagenomic"}),
]

target = Transform(NS)
target.AddRequirement({"annotations", "metapathways"})
target.AddRequirement({"taxonomy", "GTDB-TK"})

plan = Solve(have, target, transforms)[0]
_print_sol(plan)

step 1
{raw reads-dna}->{assembly-contigs}
used
  (raw reads-metagenomic-dna:000U)
produced
  (assembly-contigs:000c)

step 2
{assembly-contigs}->{orfs},{annotations-metapathways}
used
  (assembly-contigs:000c)
produced
  (orfs:000d)
  (annotations-metapathways:000e)

step 3
{contigs}->{taxonomy-GTDB-TK}
used
  (assembly-contigs:000c)
produced
  (taxonomy-GTDB-TK:000j)



In [91]:
NS = Namespace()
transforms = []

t = Transform(NS) # assembly <-> bins
t.AddRequirement({"assembly"})
t.AddProduct({"bins"})
transforms.append(t)
t = Transform(NS)
t.AddRequirement({"bins"})
t.AddProduct({"assembly"})
transforms.append(t)

t = Transform(NS) # bins <-> tax
t.AddRequirement({"bins"})
t.AddProduct({"tax"})
transforms.append(t)
t = Transform(NS)
t.AddRequirement({"tax"})
t.AddProduct({"bins"})
transforms.append(t)

t = Transform(NS) # bins <-> contigs
t.AddRequirement({"bins"})
t.AddProduct({"contigs"})
transforms.append(t)
t = Transform(NS)
t.AddRequirement({"contigs"})
t.AddProduct({"bins"})
transforms.append(t)

t = Transform(NS) # contigs <-> ORFs
t.AddRequirement({"contigs"})
t.AddProduct({"ORFs"})
transforms.append(t)
t = Transform(NS)
t.AddRequirement({"ORFs"})
t.AddProduct({"contigs"})
transforms.append(t)

t = Transform(NS) # ORFs <-> annotation
t.AddRequirement({"ORFs"})
t.AddProduct({"annotation"})
transforms.append(t)
t = Transform(NS) 
t.AddRequirement({"annotation"})
t.AddProduct({"ORFs"})
transforms.append(t)


have = [
    Endpoint(NS, {"annotation"}),
]

target = Transform(NS)
tax = target.AddRequirement({"tax"})
target.AddRequirement({"annotation"}, {tax})

plan = Solve(have, target, transforms)[0]
_print_sol(plan)

step 1
{annotation}->{ORFs}
used
  (annotation:000f)
produced
  (ORFs:000n)

step 2
{ORFs}->{contigs}
used
  (ORFs:000n)
produced
  (contigs:000o)

step 3
{contigs}->{bins}
used
  (contigs:000o)
produced
  (bins:000p)

step 4
{bins}->{tax}
used
  (bins:000p)
produced
  (tax:000t)

step 5
{tax}->{bins}
used
  (tax:000t)
produced
  (bins:000u)

step 6
{bins}->{contigs}
used
  (bins:000u)
produced
  (contigs:000x)

step 7
{contigs}->{ORFs}
used
  (contigs:000x)
produced
  (ORFs:0010)

step 8
{ORFs}->{annotation}
used
  (ORFs:0010)
produced
  (annotation:0016)



In [92]:
NS = Namespace()
transforms = []

for _destination, adjacents in [
    ("town",    ["field"]),
    ("field",   ["town", "castle"]),
    ("castle",  ["field"]),
]:
    for _source in adjacents:
        t = Transform(NS)
        t.AddRequirement(_set(_source))
        t.AddProduct(_set(_destination))
        transforms.append(t)

start = [
    Endpoint(NS, {"town"}),
]

target = Transform(NS)
target.AddRequirement({"castle"})

plan = Solve(start, target, transforms)[0]
_print_sol(plan)

step 1
{town}->{field}
used
  (town:000H)
produced
  (field:000O)

step 2
{field}->{castle}
used
  (field:000O)
produced
  (castle:000P)



In [93]:
transforms

[{field}->{town}, {town}->{field}, {castle}->{field}, {field}->{castle}]

In [94]:

NS = Namespace()
transforms = []

t = Transform(NS)
t.AddRequirement(_set("reads"))
t.AddProduct(_set("annable, taxable"))
transforms.append(t)

t = Transform(NS)
t.AddRequirement(_set("annable"))
t.AddProduct(_set("ann"))
transforms.append(t)

# t = Transform(NS)
# t.AddRequirement(_set("ann"))
# t.AddProduct(_set("annable"))
# transforms.append(t)

t = Transform(NS)
t.AddRequirement(_set("taxable"))
t.AddProduct(_set("tax"))
transforms.append(t)

t = Transform(NS)
d_parent = t.AddRequirement(_set("annable, taxable"))
d_ann = t.AddRequirement(_set("ann"), {d_parent})
d_tax = t.AddRequirement(_set("tax"), {d_parent})
t.AddProduct(_set("sum"))
transforms.append(t)

# M, N = 2, 1
# M, N = 2, 2
# M, N = 50, 2
# M, N = 64, 64
M, N = 128, 128
haves = [Endpoint(NS, _set(f"{i+1}, reads")) for i in range(M)]

target = Transform(NS)
# for e in haves[-N:]:
for e in haves[:N]:
    de = target.AddRequirement(e.properties)
    target.AddRequirement(_set("sum"), {de})
    # target.AddRequirement(_set("sum"))

print("Start")
# %prun r = Solve(haves, target, transforms)
%prun solutions = Solve(haves, target, transforms)
# f"input size [{N}], states checked [{r.steps}], {r.message}, {len(target.requires)}"

Start
 

         1472741 function calls (1438028 primitive calls) in 0.487 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
33536/513    0.110    0.000    0.311    0.001 3874813944.py:387(_gather)
   180864    0.058    0.000    0.134    0.000 3874813944.py:290(<genexpr>)
   282240    0.050    0.000    0.062    0.000 3874813944.py:86(Iterparents)
    50560    0.049    0.000    0.210    0.000 3874813944.py:288(_satisfies_lineage)
   216216    0.043    0.000    0.069    0.000 3874813944.py:57(IsA)
   216216    0.026    0.000    0.026    0.000 {method 'issubset' of 'set' objects}
        1    0.023    0.023    0.029    0.029 history.py:55(only_when_enabled)
    50944    0.021    0.000    0.151    0.000 {built-in method builtins.all}
   137860    0.014    0.000    0.014    0.000 3874813944.py:29(__hash__)
    32896    0.012    0.000    0.020    0.000 3874813944.py:32(__eq__)
    85272    0.012    0.000    0.012    0.000 {method 'items' o

In [95]:
print(len(solutions))
for i, res in enumerate(solutions):
    print(res.steps)
    for a in res.dependency_plan:
        print(a)
    print(res.application)
    print()
    if i > 10: break

1
512
{reads}->{annable-taxable} || (reads-1:000J)->(annable-taxable:008N)
{annable}->{ann} || (annable-taxable:008N)->(ann:00AN)
{taxable}->{tax} || (annable-taxable:008N)->(tax:00CN)
{annable-taxable},{ann},{tax}->{sum} || (annable-taxable:008N),(ann:00AN),(tax:00CN)->(sum:00EN)
{reads}->{annable-taxable} || (2-reads:000K)->(annable-taxable:008O)
{annable}->{ann} || (annable-taxable:008O)->(ann:00AO)
{taxable}->{tax} || (annable-taxable:008O)->(tax:00CO)
{annable-taxable},{ann},{tax}->{sum} || (annable-taxable:008O),(ann:00AO),(tax:00CO)->(sum:00EO)
{reads}->{annable-taxable} || (3-reads:000L)->(annable-taxable:008P)
{annable}->{ann} || (annable-taxable:008P)->(ann:00AP)
{taxable}->{tax} || (annable-taxable:008P)->(tax:00CP)
{annable-taxable},{ann},{tax}->{sum} || (annable-taxable:008P),(ann:00AP),(tax:00CP)->(sum:00EP)
{reads}->{annable-taxable} || (4-reads:000M)->(annable-taxable:008Q)
{annable}->{ann} || (annable-taxable:008Q)->(ann:00AQ)
{taxable}->{tax} || (annable-taxable:008Q)